<a href="https://colab.research.google.com/github/shuvanyu/Document-Retrieval-and-Ranking/blob/main/desm_nfcorpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

In [1]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 91.1 MB/s eta 0:00:00


In [2]:
!pip install beir


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/22

# Training

In [20]:
path = 'drive/MyDrive/retrain_nfc/'

In [21]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader
import pathlib, sys, os, glob, pickle, math, shutil
import nltk
import numpy as np
nltk.download('punkt')
import gensim
from operator import itemgetter
import re
dataset = "nfcorpus"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(path, 'nfcorpus')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
def create_working_directory():           
    CONTENT_PATH = os.path.join(path, 'inputs/contents/')
    TOKENS_PATH = os.path.join(path, 'inputs/tokens/')
    CENTROIDS_PATH = os.path.join(path, 'inputs/centroids/')
    DOC_NUMBER_PATH = os.path.join(path, 'inputs/doc_num/')
    MODEL_PATH = os.path.join(path, 'model')
            
    if os.path.exists(CONTENT_PATH):
        shutil.rmtree(CONTENT_PATH)
    os.makedirs(CONTENT_PATH)

    if os.path.exists(TOKENS_PATH):
        shutil.rmtree(TOKENS_PATH)
    os.makedirs(TOKENS_PATH)

    if os.path.exists(CENTROIDS_PATH):
        shutil.rmtree(CENTROIDS_PATH)
    os.makedirs(CENTROIDS_PATH)

    if os.path.exists(DOC_NUMBER_PATH):
        shutil.rmtree(DOC_NUMBER_PATH)
    os.makedirs(DOC_NUMBER_PATH)

    if os.path.exists(MODEL_PATH):
        shutil.rmtree(MODEL_PATH)
    os.makedirs(MODEL_PATH)
        
    return [CONTENT_PATH, TOKENS_PATH, CENTROIDS_PATH, DOC_NUMBER_PATH, MODEL_PATH]

In [23]:
def preprocessing(preprocess):
    PATH = create_working_directory()
    data_path = util.download_and_unzip(url, out_dir)
    corpus, queries, qrels = GenericDataLoader(data_path).load(split="train")

    count = 0
    if preprocess:
        CONTENT_PATH = PATH[0]
        TOKENS_PATH = PATH[1]
        CENTROIDS_PATH = PATH[2]
        DOC_NUMBER_PATH = PATH[3]

        for key in corpus.keys():
            content = corpus[key]['text']
            countWords = len(content.split())
            
            if countWords >= 30:
                title = corpus[key]['title']
                doc_num = key  
                count = count + 1
                content_name = CONTENT_PATH + str(count) + '.txt'
                token_name = TOKENS_PATH + str(count) + '.tokens'
                doc_name = DOC_NUMBER_PATH + str(count) + '.txt'

                cf = open(content_name, 'w', encoding="utf-8")
                cf.write(content)
                cf.close()

                df = open(doc_name, 'w', encoding="utf-8")
                df.write(doc_num)
                df.close()

                tf = open(token_name, 'w', encoding="utf-8")
                sentence_tokens = nltk.sent_tokenize(content)
                for sentence in sentence_tokens:
                    sentence = sentence.lower()
                    sentence = re.sub(r'[^\w\s]', '', sentence)
                    tf.write(sentence)
                tf.close()

    print("Total Document count is: ", count)


In [24]:
def Train(model_path, path):
  sent_iterator = gensim.models.word2vec.PathLineSentences(path)
  model = gensim.models.Word2Vec(sent_iterator, 
                                  vector_size=200,
                                  min_count=1, 
                                  workers=6)

  model.save(os.path.join(model_path, 'model/w2v-lc.model'))
  model.wv.save_word2vec_format(os.path.join(model_path, 'model/w2v-lc.model.bin'), binary=True)

  return model

In [25]:
preprocessing(True)

  0%|          | 0/3633 [00:00<?, ?it/s]

Total Document count is:  3624


In [26]:
path

'drive/MyDrive/retrain_nfc/'

In [28]:
model = Train('drive/MyDrive/retrain_nfc/', 'drive/MyDrive/retrain_nfc/inputs/tokens/')

# Testing beir

In [3]:
!tar -xf drive/MyDrive/nlp_datashare/Document_retrieval_and_ranking.tar

In [4]:
!ls Document-Retrieval-and-Ranking/Dual-Space-Embedding-Model-nfcorpus/model

w2v-lc.model	  w2v-lc.model.syn1neg.npy
w2v-lc.model.bin  w2v-lc.model.wv.vectors.npy


In [5]:
open('Document-Retrieval-and-Ranking/Dual-Space-Embedding-Model-nfcorpus/inputs/doc_num/1357.txt', 'r').read()

'MED-2412'

In [6]:
def rename_doc_num(k):
  num = k.split('/')[-1].split('.')[0]
  return open(f'Document-Retrieval-and-Ranking/Dual-Space-Embedding-Model-nfcorpus/inputs/doc_num/{num}.txt', 'r').read()

import numpy as np
clean_centroid_dict = np.load('Document-Retrieval-and-Ranking/Dual-Space-Embedding-Model-nfcorpus/clean_centroid_dict.npy', allow_pickle=True)
clean_centroid_dict = dict(enumerate(clean_centroid_dict.flatten(), 1))[1]
clean_centroid_dict_in_in = {rename_doc_num(k): v[0] for k,v in clean_centroid_dict.items()}
clean_centroid_dict_in_out = {rename_doc_num(k): v[1] for k,v in clean_centroid_dict.items()}

In [7]:
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

import logging
import pathlib, os

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

#### Download scifact.zip dataset and unzip the dataset
dataset = "nfcorpus"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join('drive', 'MyDrive',  "nlp_datashare")

/usr/local/lib/python3.10/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
data_path = util.download_and_unzip(url, out_dir)
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test") # or split = "train" or "dev"


  0%|          | 0/3633 [00:00<?, ?it/s]

In [9]:
list(qrels.keys())[0]

'PLAIN-2'

In [10]:
list(corpus.keys())[0]

'MED-10'

In [11]:
corpus_ids = sorted(corpus, key=lambda k: len(corpus[k].get("title", "") + corpus[k].get("text", "")), reverse=True)

In [12]:
type(corpus_ids)

list

In [13]:
corpus_ids[0]

'MED-957'

In [14]:
for id in corpus_ids:
  if id not in clean_centroid_dict_in_in:
    corpus.pop(id, None)

In [15]:
corpus_ids = sorted(corpus, key=lambda k: len(corpus[k].get("title", "") + corpus[k].get("text", "")), reverse=True)

In [16]:
clean_centroid_dict_in_in_list = [clean_centroid_dict_in_in[id] for id in corpus_ids]
clean_centroid_dict_in_out_list = [clean_centroid_dict_in_out[id] for id in corpus_ids]

In [60]:
import sys, os, csv, glob, json, uuid, pickle, math
import gensim, logging
import numpy as np, scipy, pandas as pd
import nltk
nltk.download('punkt')
from operator import itemgetter
import re, json

class DESM:
    def __init__(self, model_path, corpus_embedding, **kwargs):
      self.model = gensim.models.word2vec.Word2Vec.load(model_path)
      self.corpus_embedding = corpus_embedding


    def get_embedding(self, x, vector_size, model, out=False):
      if x in model.wv and out == True:
          return model.syn1neg[model.wv.key_to_index[x]]
      
      elif x in model.wv and out == False:
          return model.wv[x]
      
      else:
          return np.zeros(vector_size)
    
    def encode_corpus(self, corpus, **kwargs):
      return np.array(self.corpus_embedding)

    def encode_queries(self, queries, **kwargs):
      list_embed = []
      for query in queries:
        query_tokens = nltk.word_tokenize(query.lower())
        list_embed.append([self.get_embedding(x, self.model.vector_size, self.model)
              for x in query_tokens])
      return list_embed



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
ls drive/MyDrive/retrain_nfc/model/

w2v-lc.model      w2v-lc.model.syn1neg.npy
w2v-lc.model.bin  w2v-lc.model.wv.vectors.npy


In [61]:
d = DESM('drive/MyDrive/retrain_nfc/model/w2v-lc.model', clean_centroid_dict_in_in_list)

q_e = d.encode_queries(['Is veal bad?'])

In [62]:
type(q_e)

list

In [63]:
len(q_e)

1

In [64]:
len(q_e[0])

4

In [66]:
c_e = d.encode_corpus(corpus)

In [56]:
c_e.shape

(3624, 200)

generate dictionary that is top 1000 results for each query with the score. pass to pytrec_eval

In [75]:
from tqdm import tqdm

In [76]:
len(queries)

323

In [ ]:
corpus['MED-2468']

In [77]:
def score_document(query_embed, doc_centroid):
    cos_similarity = [(1 - scipy.spatial.distance.cosine(qin, doc_centroid)) 
                        for qin in query_embed]
    
    return (sum(cos_similarity)/len(query_embed))

scores_in_in_results = {}
scores_in_out_results = {}

for query_key, query_val in tqdm(queries.items(), total = 323):
  q_e = d.encode_queries([query_val])

  scores_in_in = []
  scores_in_out = []

  for k,v in clean_centroid_dict_in_in.items():
      scores_in_in.append((k, score_document(q_e[0], v)))
  for k,v in clean_centroid_dict_in_out.items():
      scores_in_out.append((k, score_document(q_e[0], v)))

  scores_in_in = dict(sorted(scores_in_in, key=itemgetter(1), reverse=True)[:1000])
  scores_in_out = dict(sorted(scores_in_out, key=itemgetter(1), reverse=True)[:1000])

  scores_in_in_results[query_key] = scores_in_in
  scores_in_out_results[query_key] = scores_in_out

  0%|          | 0/323 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 323/323 [06:24<00:00,  1.19s/it]


In [78]:
import pytrec_eval


In [85]:
query_val = 'veal'
scores_in_in = []
q_e = d.encode_queries([query_val])
for k,v in clean_centroid_dict_in_in.items():
  scores_in_in.append((k, score_document(q_e[0], v)))

scores_in_in = sorted(scores_in_in, key=itemgetter(1), reverse=True)[:1000]

query_val = 'veal'
scores_in_out = []
q_e = d.encode_queries([query_val])
for k,v in clean_centroid_dict_in_in.items():
  scores_in_out.append((k, score_document(q_e[0], v)))

scores_in_out = sorted(scores_in_out, key=itemgetter(1), reverse=True)[:1000]

In [82]:
path

'drive/MyDrive/retrain_nfc/'

In [93]:
def view_files(doc_list):
    for path in doc_list:
        print('\033[1m' + 'DOCUMENT NUMBER: ' + '\033[0m', path)
        
        print(corpus[path])

In [94]:
top_5_in_in = [x[0] for x in scores_in_in[:5]]

in_in_top5 = []

for fname in top_5_in_in:
    in_in_top5.append(fname)

view_files(in_in_top5)

DOCUMENT NUMBER:  MED-2468
{'text': 'BACKGROUND AND METHODS: We estimated the prevalence of self-reported asthma in adult Indians and examined several risk factors influencing disease prevalence. Analysis is based on 99 574 women and 56 742 men aged 20–49 years included in India’s third National Family Health Survey, 2005–2006. Multiple logistic regression analysis was used to estimate the prevalence odds ratios for asthma, adjusting for various risk factors. RESULTS: The prevalence of self-reported asthma was 1.8% (95%CI 1.6–2.0) among men and 1.9% (95%CI 1.8–2.0) among women, with higher rates in rural than in urban areas and marked geographic differences. After adjustment for known asthma risk factors, women were 1.2 times more likely to have asthma than men. Daily/weekly consumption of milk/milk products, green leafy vegetables and fruits were associated with a lower asthma risk, whereas consumption of chicken/meat, a lower body mass index (BMI; <16 kg/m2, OR 2.08, 95%CI 1.73–2.50)

In [87]:
in_in_top5[0]

'MED-2468'

In [88]:
path + '/inputs/contents/'

'drive/MyDrive/retrain_nfc//inputs/contents/'

In [ ]:
ls drive/MyDrive/retrain_nfc/inputs/contents/

In [ ]:
k_values = [1,3,5,10,100,1000]

ndcg = {}
_map = {}
recall = {}
precision = {}

for k in k_values:
    ndcg[f"NDCG@{k}"] = 0.0
    _map[f"MAP@{k}"] = 0.0
    recall[f"Recall@{k}"] = 0.0
    precision[f"P@{k}"] = 0.0

map_string = "map_cut." + ",".join([str(k) for k in k_values])
ndcg_string = "ndcg_cut." + ",".join([str(k) for k in k_values])
recall_string = "recall." + ",".join([str(k) for k in k_values])
precision_string = "P." + ",".join([str(k) for k in k_values])
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {map_string, ndcg_string, recall_string, precision_string})
scores_in_in = evaluator.evaluate(scores_in_in_results)

for query_id in scores_in_in.keys():
    for k in k_values:
        ndcg[f"NDCG@{k}"] += scores_in_in[query_id]["ndcg_cut_" + str(k)]
        _map[f"MAP@{k}"] += scores_in_in[query_id]["map_cut_" + str(k)]
        recall[f"Recall@{k}"] += scores_in_in[query_id]["recall_" + str(k)]
        precision[f"P@{k}"] += scores_in_in[query_id]["P_"+ str(k)]

for k in k_values:
    ndcg[f"NDCG@{k}"] = round(ndcg[f"NDCG@{k}"]/len(scores_in_in), 5)
    _map[f"MAP@{k}"] = round(_map[f"MAP@{k}"]/len(scores_in_in), 5)
    recall[f"Recall@{k}"] = round(recall[f"Recall@{k}"]/len(scores_in_in), 5)
    precision[f"P@{k}"] = round(precision[f"P@{k}"]/len(scores_in_in), 5)

for eval in [ndcg, _map, recall, precision]:
    print("\n")
    for k in eval.keys():
        print("{}: {:.4f}".format(k, eval[k]))


In [ ]:
k_values = [1,3,5,10,100,1000]

ndcg = {}
_map = {}
recall = {}
precision = {}

for k in k_values:
    ndcg[f"NDCG@{k}"] = 0.0
    _map[f"MAP@{k}"] = 0.0
    recall[f"Recall@{k}"] = 0.0
    precision[f"P@{k}"] = 0.0

map_string = "map_cut." + ",".join([str(k) for k in k_values])
ndcg_string = "ndcg_cut." + ",".join([str(k) for k in k_values])
recall_string = "recall." + ",".join([str(k) for k in k_values])
precision_string = "P." + ",".join([str(k) for k in k_values])
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {map_string, ndcg_string, recall_string, precision_string})
scores_in_out = evaluator.evaluate(scores_in_out_results)

for query_id in scores_in_out.keys():
    for k in k_values:
        ndcg[f"NDCG@{k}"] += scores_in_out[query_id]["ndcg_cut_" + str(k)]
        _map[f"MAP@{k}"] += scores_in_out[query_id]["map_cut_" + str(k)]
        recall[f"Recall@{k}"] += scores_in_out[query_id]["recall_" + str(k)]
        precision[f"P@{k}"] += scores_in_out[query_id]["P_"+ str(k)]

for k in k_values:
    ndcg[f"NDCG@{k}"] = round(ndcg[f"NDCG@{k}"]/len(scores_in_out), 5)
    _map[f"MAP@{k}"] = round(_map[f"MAP@{k}"]/len(scores_in_out), 5)
    recall[f"Recall@{k}"] = round(recall[f"Recall@{k}"]/len(scores_in_out), 5)
    precision[f"P@{k}"] = round(precision[f"P@{k}"]/len(scores_in_out), 5)

for eval in [ndcg, _map, recall, precision]:
    print("\n")
    for k in eval.keys():
        print("{}: {:.4f}".format(k, eval[k]))